In [1]:
# Data cleaning - to make column for each element so that we could easily manipulate data later on

import numpy as np
import pandas as pd
import seaborn as sns

# Loads the food balance data
total = pd.read_csv("food_balance_total.csv",encoding = "ISO-8859-1", engine='python')
item_group = pd.read_csv("food_balance_item_group.csv")
total=total.merge(item_group, on='Item Code', how='left')
animal=total.loc[total['Item Group Code'] ==2941]
vegetal=total.loc[total['Item Group Code'] ==2903]
animal.to_csv('food_balance_animal.csv')
vegetal.to_csv('food_balance_vegetal.csv')

In [2]:
# Loads the food balance data
ani = pd.read_csv("food_balance_animal.csv")
veg = pd.read_csv("food_balance_vegetal.csv")

ani=ani.drop(['Unnamed: 0','Item Group Code', 'Item Group', 'Item_y', 'Factor', 'HS Code','HS07 Code', 'HS12 Code', 'CPC Code'], axis=1)
veg=veg.drop(['Unnamed: 0','Item Group Code', 'Item Group', 'Item_y', 'Factor', 'HS Code','HS07 Code', 'HS12 Code', 'CPC Code'], axis=1)
# Adds variable ‘origin’
veg["origin"] = "vegetal"
ani["origin"] = "animal"

In [3]:
# Adds variable ‘origin’
veg["origin"] = "vegetal"
ani["origin"] = "animal"
# Appends veg and ani to one dataframe
temp = ani.append(veg)
# Deletion of ani and veg file
del ani, veg
# Renaming of temp’s columns
temp.columns = ["country_code", "country",'item_code','item','x1','element','x2','year','unit','value','x3','origin']


In [4]:
# Transformation of ‘temp’ to a pivot table
data = temp.pivot_table(
index=["country_code", "country", "item_code", "item", "year", "origin"],
columns = ["element"], values=["value"], aggfunc=sum)


In [5]:

# Renaming of data’s columns (be careful, the order of the columns needs
# to be adapted to your own data !)
data.columns = ['domestic_supply_quantity', 'export_quantity','fat_supply_quantity_gcapitaday','feed',
'food', 'food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr', 'import_quantity','losses', 'other_uses',
'processing', 'production', 'protein_supply_quantity_gcapitaday','residuals','seed', 'stock_variation', 'tourist_consumption']
# Index columns need to be normal columns
data = data.reset_index()
data.columns

Index(['country_code', 'country', 'item_code', 'item', 'year', 'origin',
       'domestic_supply_quantity', 'export_quantity',
       'fat_supply_quantity_gcapitaday', 'feed', 'food',
       'food_supply_kcalcapitaday', 'food_supply_quantity_kgcapitayr',
       'import_quantity', 'losses', 'other_uses', 'processing', 'production',
       'protein_supply_quantity_gcapitaday', 'residuals', 'seed',
       'stock_variation', 'tourist_consumption'],
      dtype='object')

In [6]:
data.head()

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,fat_supply_quantity_gcapitaday,feed,...,import_quantity,losses,other_uses,processing,production,protein_supply_quantity_gcapitaday,residuals,seed,stock_variation,tourist_consumption
0,1,Armenia,2511,Wheat and products,2014,vegetal,639.0,1.0,3.90,110.0,...,344.0,30.0,0.0,16.0,338.0,33.01,46.0,32.0,42.0,NaN
1,1,Armenia,2511,Wheat and products,2015,vegetal,748.0,2.0,4.02,125.0,...,324.0,59.0,0.0,16.0,363.0,33.07,106.0,33.0,-63.0,NaN
2,1,Armenia,2511,Wheat and products,2016,vegetal,650.0,1.0,4.22,128.0,...,312.0,64.0,0.0,17.0,350.0,34.70,-22.0,33.0,11.0,NaN
3,1,Armenia,2511,Wheat and products,2017,vegetal,547.0,1.0,4.28,105.0,...,337.0,15.0,0.0,16.0,176.0,34.36,-42.0,25.0,-34.0,NaN
4,1,Armenia,2511,Wheat and products,2018,vegetal,571.0,1.0,4.40,118.0,...,379.0,16.0,0.0,16.0,187.0,34.25,-27.0,20.0,-5.0,NaN


In [7]:
## calculate population in the world

population=total.loc[(total['Element'] == 'Total Population - Both sexes') & (total['Year'] == 2018)]
population=population[['Area', 'Value']]
population['Value']=population['Value']*1000
population = population.rename(columns = {'Area': 'country', 'Value': 'population'}, inplace = False)


population=population.set_index('country')
population=population.drop(['Australia', 'China, Hong Kong SAR','China, Macao SAR','China, mainland','China, Taiwan Province of'
                 ,'Northern Africa','World','Northern Africa','Southern Africa','Western Africa','Northern America'
                ,'Central America','South America','Asia','Central Asia','Eastern Asia','Southern Asia','South-eastern Asia'
                ,'Western Asia','Europe','Eastern Europe','Northern Europe','Southern Europe','Western Europe'
                ,'European Union (28)','European Union (27)','Least Developed Countries','Land Locked Developing Countries'
                ,'Low Income Food Deficit Countries','Net Food Importing Developing Countries','Americas'
                ,'Small Island Developing States','Middle Africa','Eastern Africa','Africa'])
population=population.reset_index()
population.population.sum()

7531624000.0

In [8]:
population.head()

,country,population
0,Afghanistan,37172000.0
1,Albania,2883000.0
2,Algeria,42228000.0
3,Angola,30810000.0
4,Antigua and Barbuda,96000.0


In [9]:
## Creating new variables based on existing variables: food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,great_import_from_undern_countries

df=pd.merge(data,population,on='country')
pd.set_option('float_format', '{:f}'.format)
df['food_supply_kcal']=df['food_supply_kcalcapitaday']*df['population']*365
df['food_supply_kgprotein']=df['protein_supply_quantity_gcapitaday']*365*0.001*df['population']
df['food_supply_kg']=df['food']*1000000
df['ratio_kcalkg']=df['food_supply_kcal']/df['food_supply_kg']
df['protein_percentage']=df['food_supply_kgprotein']/df['food_supply_kg']
df['dom_sup_kcal']=df['ratio_kcalkg']*df['domestic_supply_quantity']*1000000
df['dom_sup_kgprot']=df['protein_percentage']*df['domestic_supply_quantity']*1000000

egg=df.loc[df['item'] == 'Eggs']
egg

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,fat_supply_quantity_gcapitaday,feed,...,stock_variation,tourist_consumption,population,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot
377,1,Armenia,2744,Eggs,2014,animal,36.000000,nan,3.150000,nan,...,0.000000,nan,2952000.000000,48486600000.000000,3695756.400000,34000000.000000,1426.076471,0.108699,51338752941.176468,3913153.835294
378,1,Armenia,2744,Eggs,2015,animal,37.000000,nan,3.170000,nan,...,0.000000,nan,2952000.000000,48486600000.000000,3717306.000000,34000000.000000,1426.076471,0.109333,52764829411.764702,4045303.588235
379,1,Armenia,2744,Eggs,2016,animal,35.000000,0.000000,3.020000,nan,...,0.000000,nan,2952000.000000,46331640000.000000,3544909.200000,33000000.000000,1403.989091,0.107421,49139618181.818176,3759752.181818
380,1,Armenia,2744,Eggs,2017,animal,35.000000,nan,3.020000,nan,...,0.000000,nan,2952000.000000,46331640000.000000,3544909.200000,33000000.000000,1403.989091,0.107421,49139618181.818176,3759752.181818
381,1,Armenia,2744,Eggs,2018,animal,36.000000,nan,3.100000,nan,...,0.000000,nan,2952000.000000,47409120000.000000,3652657.200000,34000000.000000,1394.385882,0.107431,50197891764.705887,3867519.388235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81767,5504,Polynesia,2744,Eggs,2014,animal,4.000000,nan,1.840000,nan,...,0.000000,0.000000,474000.000000,4498260000.000000,352940.400000,3000000.000000,1499.420000,0.117647,5997680000.000000,470587.200000
81768,5504,Polynesia,2744,Eggs,2015,animal,4.000000,0.000000,1.720000,nan,...,0.000000,0.000000,474000.000000,4152240000.000000,332179.200000,3000000.000000,1384.080000,0.110726,5536320000.000000,442905.600000
81769,5504,Polynesia,2744,Eggs,2016,animal,3.000000,nan,1.510000,nan,...,0.000000,0.000000,474000.000000,3633210000.000000,290656.800000,3000000.000000,1211.070000,0.096886,3633210000.000000,290656.800000
81770,5504,Polynesia,2744,Eggs,2017,animal,3.000000,0.000000,1.520000,nan,...,0.000000,0.000000,474000.000000,3806220000.000000,294117.000000,3000000.000000,1268.740000,0.098039,3806220000.000000,294117.000000


In [10]:
food_security = pd.read_csv("food_security.csv")
food_security.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FS,Suite of Food Security Indicators,2,Afghanistan,6121,Value,21010,Average dietary energy supply adequacy (percen...,20132015,2013-2015,%,96,F,FAO estimate,nan
1,FS,Suite of Food Security Indicators,2,Afghanistan,6122,Value,21011,Average value of food production (constant 200...,20132015,2013-2015,I$ per person,104,F,FAO estimate,nan
2,FS,Suite of Food Security Indicators,2,Afghanistan,6128,Value,22000,Dietary energy supply used in the estimation o...,20132015,2013-2015,kcal/capita/day,2040,F,FAO estimate,nan
3,FS,Suite of Food Security Indicators,2,Afghanistan,6121,Value,21012,Share of dietary energy supply derived from ce...,20132015,2013-2015,%,74,F,FAO estimate,nan
4,FS,Suite of Food Security Indicators,2,Afghanistan,6123,Value,21013,Average protein supply (g/cap/day) (3-year ave...,20132015,2013-2015,g/capita/day,56,F,FAO estimate,nan


In [11]:
## find a country with malnutrition rate over 10percent
malnutrition_data=food_security.loc[(food_security['Item'] == 'Prevalence of undernourishment (percent) (3-year average)')]
malnutrition_data=malnutrition_data.replace({'Value': {'<': ''}}, regex=True)
malnutrition_data = malnutrition_data[malnutrition_data['Value'].notna()]
#malnutrition_data.sort_values("Value")
malnutrition_data.Value= malnutrition_data.Value.astype(float)
malnutrition_data=malnutrition_data.loc[(malnutrition_data['Value'] >10)]
malnutrition_data=malnutrition_data.drop(['Domain Code','Domain', 'Area Code', 'Element Code', 'Element', 'Item Code','Year Code', 'Year', 'Unit','Flag','Flag Description','Note'], axis=1)
malnutrition_data = malnutrition_data.rename(columns = {'Area': 'country'}, inplace = False)
malnutrition_data.head()

,country,Item,Value
8,Afghanistan,Prevalence of undernourishment (percent) (3-ye...,26.400000
93,Angola,Prevalence of undernourishment (percent) (3-ye...,23.100000
291,Bangladesh,Prevalence of undernourishment (percent) (3-ye...,14.500000
459,Bolivia (Plurinational State of),Prevalence of undernourishment (percent) (3-ye...,17.300000
505,Botswana,Prevalence of undernourishment (percent) (3-ye...,19.400000


In [12]:
exported=df.loc[(df['year'] ==2018)]
exported=exported[["country", "item","export_quantity","year"]]
exported = exported[exported['export_quantity'].notna()]
exported

,country,item,export_quantity,year
4,Armenia,Wheat and products,1.000000,2018
14,Armenia,Maize and products,0.000000,2018
39,Armenia,"Cereals, Other",0.000000,2018
44,Armenia,Potatoes and products,0.000000,2018
64,Armenia,Sugar beet,0.000000,2018
...,...,...,...,...
81816,Polynesia,"Aquatic Animals, Others",0.000000,2018
81821,Polynesia,Aquatic Plants,0.000000,2018
81831,Polynesia,Rice and products,0.000000,2018
81836,Polynesia,Milk - Excluding Butter,0.000000,2018


In [13]:
## finding 25 most exported items by countries with mal-nutrition rate over 10% - using left join
exported_items_top_25= pd.merge(malnutrition_data,exported,on='country',how='left')
exported_items_top_25=exported_items_top_25.groupby('item')['export_quantity'].sum()
exported_items_top_25= pd.DataFrame(data=exported_items_top_25)
exported_items_top_25=exported_items_top_25.reset_index()
exported_items_top_25=exported_items_top_25.nlargest(25,['export_quantity'])
exported_items_top_25

,item,export_quantity
73,Rice and products,25213.000000
4,Bananas,14008.000000
83,Sugar (Raw Equivalent),9210.000000
93,Wheat and products,3730.000000
92,"Vegetables, Other",3298.000000
62,Pelagic Fish,2455.520000
41,Maize and products,2417.000000
17,Cocoa Beans and products,2151.000000
66,Pineapples and products,2042.000000
32,"Fruits, Other",2038.000000


In [14]:
import_items=df.loc[(df['year'] ==2018)]
#import_items=import_items[["country", "item","import_quantity","year"]]
import_items_top_25=import_items.merge(exported_items_top_25, on='item', how='left', indicator=True)
import_items_top_25=import_items_top_25.nlargest(200,['import_quantity'])



import_items_top_25=import_items_top_25.replace({'_merge': {'both': 'True', 'left_only': 'False'}})
import_items_top_25

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity_x,fat_supply_quantity_gcapitaday,feed,...,population,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,export_quantity_y,_merge
15754,351,China,2555,Soyabeans,2018,vegetal,108661.000000,199.000000,1.150000,9111.000000,...,1459378000.000000,18110880980000.000000,1704553504.000000,5270000000.000000,3436.599806,0.323445,373424371568838.687500,35145823206.478943,nan,False
15741,351,China,2532,Cassava and products,2018,vegetal,29928.000000,203.000000,0.010000,15602.000000,...,1459378000.000000,3196037820000.000000,26633648.500000,2834000000.000000,1127.747996,0.009398,33751242017275.937500,281260350.143966,1088.000000,True
8808,138,Mexico,2514,Maize and products,2018,vegetal,40514.000000,1223.000000,11.030000,18622.000000,...,126191000.000000,45829416425000.000000,1180971092.600000,14813000000.000000,3093.864607,0.079725,125344830692125.171875,3229991416.026221,2417.000000,True
6752,110,Japan,2514,Maize and products,2018,vegetal,15819.000000,1.000000,0.260000,12010.000000,...,127202000.000000,3760727130000.000000,9750033.300000,1685000000.000000,2231.885537,0.005786,35306197311258.156250,91534585.621780,2417.000000,True
14789,237,Viet Nam,2514,Maize and products,2018,vegetal,14655.000000,82.000000,1.160000,12259.000000,...,95546000.000000,4324411960000.000000,107412813.200000,1402000000.000000,3084.459315,0.076614,45202751265192.585938,1122778015.296719,2417.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14391,231,United States of America,2761,Freshwater Fish,2018,animal,1746.680000,373.070000,0.290000,0.000000,...,327096000.000000,955120320000.000000,145655848.800000,1646680000.000000,580.027886,0.088454,1013123108641.387573,154501274.067812,nan,False
1795,33,Canada,2542,Sugar (Raw Equivalent),2018,vegetal,1246.000000,220.000000,0.000000,0.000000,...,37075000.000000,4682201750000.000000,0.000000,1246000000.000000,3757.786316,0.000000,4682201750000.000000,0.000000,9210.000000,True
11665,185,Russian Federation,2605,"Vegetables, Other",2018,vegetal,12380.000000,186.000000,0.340000,2246.000000,...,145734000.000000,2500066770000.000000,110109323.700000,9747000000.000000,256.496026,0.011297,3175420807694.675293,139853639.828255,3298.000000,True
13451,216,Thailand,2763,Pelagic Fish,2018,animal,909.400000,1091.600000,1.510000,150.000000,...,69428000.000000,886942700000.000000,127466336.600000,759400000.000000,1167.951936,0.167851,1062135490360.811157,152644043.328997,2455.520000,True


In [15]:
# Identify major trends: considering only plant products, what portion of global domestic supply is used as food,feed,losses,other uses

plant_data_2018=df[(df['year'] ==2018)& (df['origin'] =='vegetal')]
plant_data_2018=plant_data_2018.fillna(0)
##plant_data_2018.to_csv('plant_data_2018.csv')
global_supply_quantity= plant_data_2018['domestic_supply_quantity'].sum()
global_feed_food_losses_others=plant_data_2018['food']+plant_data_2018['feed']+plant_data_2018['losses']+plant_data_2018['other_uses']
plant_data_2018['global_feed_food_losses_others']=global_feed_food_losses_others
global_feed_food_losses_others=plant_data_2018['global_feed_food_losses_others'].sum()
global_supply_quantity/global_feed_food_losses_others

1.406293416460277

In [16]:
global_supply_quantity= plant_data_2018['domestic_supply_quantity'].sum()
global_feed_food_losses_others=plant_data_2018['food']+plant_data_2018['feed']+plant_data_2018['losses']+plant_data_2018['other_uses']
plant_data_2018['global_feed_food_losses_others']=global_feed_food_losses_others
global_feed_food_losses_others=plant_data_2018['global_feed_food_losses_others'].sum()
global_supply_quantity/global_feed_food_losses_others

1.406293416460277

In [29]:
# 2.How many humans on earth could be fed if all the plant-based food supply (crops), 
# including food and feed, was used for human consumption? 
# Give the results in terms of calories, and protein. Express these two results as a percentage of the world's population.
# assume that human need 2000kcal a day and 50g(0.05) of protein intake/day
plant_feed_capacity=plant_data_2018
plant_feed_capacity=plant_feed_capacity.groupby(['country', 'item'])['food','feed','ratio_kcalkg','protein_percentage','population'].sum().reset_index()
plant_feed_capacity['sum_food_feed']=plant_feed_capacity['food']+plant_feed_capacity['feed']
plant_feed_capacity['feedable_population_kcal']=plant_feed_capacity['sum_food_feed']*1000000*plant_feed_capacity['ratio_kcalkg']/2000
plant_feed_capacity['feedable_population_protein']=plant_feed_capacity['sum_food_feed']*1000000*plant_feed_capacity['protein_percentage']/0.05
plant_feed_capacity['feedable_percentage_kcal']=plant_feed_capacity['feedable_population_kcal']/plant_feed_capacity['population']
plant_feed_capacity['feedable_percentage_protein']=plant_feed_capacity['feedable_population_protein']/plant_feed_capacity['population']
## I got result as INF?
plant_feed_capacity.feedable_population_kcal.sum()

inf

In [18]:
# what portion of world's population is considered undernourished?
malnutiriton_population=pd.merge(malnutrition_data,population,on='country')
mulnutrition_population_sum=malnutiriton_population.population.sum()
population_sum=population.population.sum()
malnutrition_population_percentage=mulnutrition_population_sum/population_sum
malnutrition_population_percentage

0.3416946995760808

In [19]:
#Considering the 25 items most exported by the countries with a high rate of undernutrition, which three of them:
#Have the greatest other_uses to domestic_supply_quantity ratio and what are they used for?
#Have the greatest feed to (food+feed) ratio and what are they used for?


group_item=df.groupby(['item'])['food','feed','domestic_supply_quantity','other_uses'].sum().reset_index()
top_three=pd.merge(group_item,exported_items_top_25,on='item')
top_three['ratio_other_domsupply']=top_three['other_uses']/top_three['domestic_supply_quantity']
top_three['ratio_feed_foofeed']=top_three['food']/(top_three['food']+top_three['feed'])
ratio_other_domsupply=top_three.sort_values(by='ratio_other_domsupply', ascending=False)
ratio_feed_foofeed=top_three.sort_values(by='ratio_feed_foofeed', ascending=False)
ratio_other_domsupply.head(3)


,item,food,feed,domestic_supply_quantity,other_uses,export_quantity,ratio_other_domsupply,ratio_feed_foofeed
0,"Alcohol, Non-Food",23.000000,0.000000,75628.000000,76140.000000,1330.000000,1.006770,1.000000
15,Palm Oil,79694.000000,204.000000,301744.000000,162756.000000,2028.000000,0.539384,0.997447
6,Coconut Oil,6200.000000,0.000000,14187.000000,3865.000000,975.000000,0.272433,1.000000


In [20]:
ratio_feed_foofeed.head(3).head()

,item,food,feed,domestic_supply_quantity,other_uses,export_quantity,ratio_other_domsupply,ratio_feed_foofeed
0,"Alcohol, Non-Food",23.000000,0.000000,75628.000000,76140.000000,1330.000000,1.006770,1.000000
13,Nuts and products,75637.000000,0.000000,77820.000000,180.000000,1560.000000,0.002313,1.000000
17,Pineapples and products,109176.000000,0.000000,120761.000000,0.000000,2042.000000,0.000000,1.000000


In [21]:
# Taking only grains (cereals) for food and feed into account, what proportion (in terms of weight) is used for feed?
cereal=df[(df['item'] =='Cereals, Other')]
cereal=cereal[["feed", "food"]]
cereal['feed_and_food']=cereal['food']+cereal['feed']
cereal_feed_and_food=cereal['feed_and_food'].sum()
cereal_feed=cereal['feed'].sum()
cereal_feed/cereal_feed_and_food

0.7478258288563712

In [22]:
# 7.How many tons of grains (cereals) could be released if the US reduced its production of animal products by 10%? 
# Convert this quantity to kcal, and the number of potentially fed humans.

pd.set_option('float_format', '{:f}'.format)
us_animal_production=df[(df['country'] =='United States of America')& (df['origin'] =='animal')]
us_animal_production=us_animal_production[["country", "item","food_supply_kcal","production",'population','feed']]
us_animal_production=us_animal_production.fillna(0)


us_cereals_production=df[(df['country'] =='United States of America')& (df['item'] =='Cereals, Other')]
us_cereals_production=us_cereals_production[["year","country", "item","food_supply_kcal","production",'population','feed','ratio_kcalkg']]
us_cereals_production
#us_animal_production=df.groupby(['item'])['food','feed','domestic_supply_quantity','other_uses'].sum().reset_index()

,year,country,item,food_supply_kcal,production,population,feed,ratio_kcalkg
71506,2014,United States of America,"Cereals, Other",716340240000.000000,275.000000,327096000.000000,86.000000,3673.539692
71507,2015,United States of America,"Cereals, Other",835730280000.000000,227.000000,327096000.000000,77.000000,3467.760498
71508,2016,United States of America,"Cereals, Other",835730280000.000000,226.000000,327096000.000000,65.000000,3482.209500
71509,2017,United States of America,"Cereals, Other",835730280000.000000,252.000000,327096000.000000,68.000000,3411.144000
71510,2018,United States of America,"Cereals, Other",477560160000.000000,238.000000,327096000.000000,73.000000,3363.099718


In [23]:
reduced_animal_production_kg=us_animal_production.production.sum()*0.1*1000
release_cereals_kg=reduced_animal_production_kg*0.7478258288563712
feed_quantity_kcal=release_cereals_kg*us_cereals_production.ratio_kcalkg.sum()
feed_quantity_kcal
possible_fed_human=feed_quantity_kcal/2000
possible_fed_human

516264557.64682025

In [24]:
## 8.In Thailand, what proportion of cassava is exported? What is the proportion of undernutrition?
thailand_data=df[(df['country'] =='Thailand')& (df['item'] =='Cassava and products')]
cassava_export_ratio=thailand_data.export_quantity.sum()/thailand_data.production.sum()
cassava_export_ratio


0.904748246460327

In [25]:
thailand_data

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,fat_supply_quantity_gcapitaday,feed,...,stock_variation,tourist_consumption,population,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot
66808,216,Thailand,2532,Cassava and products,2014,vegetal,5471.000000,26009.000000,0.020000,2124.000000,...,nan,nan,69428000.000000,354777080000.000000,1267061.000000,316000000.000000,1122.712278,0.004010,6142358875569.620117,21936995.984177
66809,216,Thailand,2532,Cassava and products,2015,vegetal,5659.000000,30995.000000,0.020000,2037.000000,...,200.000000,nan,69428000.000000,354777080000.000000,1267061.000000,323000000.000000,1098.381053,0.003923,6215738376842.105469,22199065.631579
66810,216,Thailand,2532,Cassava and products,2016,vegetal,6647.000000,30080.000000,0.010000,4076.000000,...,1116.000000,nan,69428000.000000,228070980000.000000,760236.600000,216000000.000000,1055.884167,0.003520,7018462055833.333984,23394873.519444
66811,216,Thailand,2532,Cassava and products,2017,vegetal,4757.000000,29409.000000,0.010000,2063.000000,...,-1316.000000,nan,69428000.000000,152047320000.000000,253412.200000,153000000.000000,993.773333,0.001656,4727379746666.666016,7878966.244444
66812,216,Thailand,2532,Cassava and products,2018,vegetal,12646.000000,22299.000000,0.000000,9677.000000,...,nan,nan,69428000.000000,76023660000.000000,253412.200000,90000000.000000,844.707333,0.002816,10682168937333.332031,35607229.791111


In [26]:
thailand_malnutrition=food_security[(food_security['Area'] =='Thailand')& (food_security['Item'] =='Prevalence of undernourishment (percent) (3-year average)')]     
thailand_malnutrition

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
3819,FS,Suite of Food Security Indicators,216,Thailand,6121,Value,210041,Prevalence of undernourishment (percent) (3-ye...,20132015,2013-2015,%,8.8,F,FAO estimate,nan
